In [ ]:
#I added some codes to the one by Jessy
#I guess If I try to work on the latest version in the git -repository, everyone can see and work on the latest version.

In [2]:

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")

import string
import re



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from gensim.models.doc2vec import TaggedDocument
from sklearn.tree import DecisionTreeClassifier

In [4]:
# Import required packages
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# load English language model of spacy
sp = spacy.load('en_core_web_sm')

## Loading 

Below is how the dataset looks like

In [5]:
df_train = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/project/training_data.csv")
df_test=pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/project/test_data.csv")

In [ ]:
#df_train = pd.read_csv("/Users/MBP/Documents/UNIL/DM&ML/Project/82653de5-2ed1-4782-959e-23eba75d67a9_training_data (1).csv")
#df_train

There are 6471 rows and 5 columns in the training data.
Since column text contains the tweets, our first assumption is that it might be the most important predictor

### What does each column show?

This dataset contains tweets about whether or not a reported disaster is real. 
Some of the features in the dataset are:

>	1.	`Id` : id of the tweet.
>	2.	`keyword`: a kind of a title for that paritucular disaster 
>	3.	`location`: the geographical location where that disaster happened.
>	4.	`text`: the disaster messages in form a tweet.
>	5.	`target`: value representing whether the reported tweet was true or not
    
Our task is to predict whether a tweet is real or not, i.e, we should use the target column as the labels.




### A look at the Test data

In [ ]:
#df_test =  pd.read_csv("/Users/MBP/Documents/UNIL/DM&ML/Project/5fc21f33-b209-4b07-ad70-a69020dfd2cf_test_data.csv")
#df_test

And we observe that there are 1142 rows and 4 columns in the training data. 
Having four columns instead 5 here indicates that these are predictors and the column "target" is the predicted 

# Initial EDA

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_train_copy = df_train.copy()

#### A quick look at the different types of tweets

Below is a tweet on real disaster 

In [ ]:
# a quick look on data
quick_look_real = df_train[df_train["target"] == 1 ]['text'].values[2]
quick_look_real


This is a typical tweet on fake disaster 

In [ ]:
# a quick look on data
quick_look_fake = df_train[df_train["target"] == 0 ]['text'].values[2]
quick_look_fake


#### Let's see how the real and fake disaster tweets are distributed

In [ ]:
df_train['target'].value_counts()


In [ ]:
import seaborn as sns

plt.subplots(1,1,figsize=(10,8))
sns.countplot('target',data=df_train) #this gives the frequency of each
plt.title("Disaster tweets Distribution")
plt.show()

It is clear that there are more fake than real ones. 

### The Base rate is given as follows:

In [ ]:
base_rate=round(df_train.target.value_counts()[0]/len(df_train), 4 )
print(f'\nThe base rate is {base_rate}')

## Let's do further exploration

In [ ]:
df_train.head(3)

#### Words Distribution

In [ ]:
# let's split the words

df_train['totalwords'] = df_train['text'].str.split().str.len()
# df_train['totalwords'] = df_train['text'].apply(lambda x: len(x.split()))

df_test['totalwords'] = df_test['text'].str.split().str.len()

df_train['totalwords'].hist(edgecolor='black', linewidth=1.2)
df_test['totalwords'].hist(edgecolor='black', linewidth=1.2)
plt.title('Total number of Words Distribution')
plt.show()

In [ ]:
df_train

This is how the total words are distributed

In [ ]:
df_train['totalwords'][df_train['target'] == 0].plot.hist(bins=20)
df_train['totalwords'][df_train['target'] == 1].plot.hist(bins=20)

Here, we see that the most longest real disaster tweets have between 15 and 20 words.

generally speaking, fake ones are more longer than the real ones. othervise, there is a similar central tendency.

#### Character Distribution Comparison

Here, we compare the distribution of real and fake disaster tweets 

In [ ]:
df_train['totalcharacter'] = df_train['text'].apply(lambda x: len(x))
df_train['totalcharacter'][df_train['target'] == 0]

#Test
df_test['totalcharacter'] = df_test['text'].apply(lambda x: len(x))


In [ ]:
df_train['totalcharacter'][df_train['target'] == 0].plot.hist()

In [ ]:
df_train['totalcharacter'][df_train['target'] == 1].plot.hist()

In [ ]:
df_train['totalcharacter'][df_train['target'] == 0].plot.hist()
df_train['totalcharacter'][df_train['target'] == 1].plot.hist()

> We observe that real disaster tweet characters are fewer than the fakes ones.
> Real disaster tweets have at least 20 characters. And most of them have at least 80 characters
> This could help latter.

#### unique word

In [ ]:
df_train['uniqueword'] = df_train['text'].apply(lambda x: len(set(x.split())))
df_train['uniqueword'][df_train['target'] == 0].plot.hist()
df_train['uniqueword'][df_train['target'] == 1].plot.hist()

If this feature 'unique word' is not too correlated with the other feature, we believe that it can be a good predictor

#### Average Word Length

In [ ]:
def avg_word_length(x):
    x = x.split()
    return np.mean([len(i) for i in x])

df_train['avg_word_length'] = df_train['text'].apply(avg_word_length)
df_train['avg_word_length'][df_train['target'] == 0].plot.hist()
df_train['avg_word_length'][df_train['target'] == 1].plot.hist()


We may use this feature during our feature engineering

#### The location with the most fake disaster tweet 

In [ ]:
df_train['location'][df_train['target'] == 0].value_counts().plot(figsize=(15, 12))


>According this most of the fake disaster tweets apparently come from New york.
>And many have a pseudoname as location

#### The location with the most real disaster tweet 

In [ ]:
df_train['location'][df_train['target'] == 1].value_counts().plot(figsize=(15, 12))


This gives an idea about the locations where most of the real tweets come from  

###### All these above can become new features to help us explore the attributes of tweets that are about disasters. 
###### If we think these new features are useful, we can add them as input our model with the text data.




# Data Cleaning

In [6]:
# A function to do some text cleaning

def func_textcleaning(sentence):
    # remove unicode literals
    temp = sentence.encode('ascii',errors='ignore').decode('ascii')
    
    # remove &amp
    temp = temp.replace('&amp;', '')
    
    # remove urls
    temp = re.sub(r"http\S+", "", temp)
    
    # remove html
    temp = re.sub(r'<.*?>', "", temp)
    
    # remove hashtags
    temp = re.sub(r'#', "", temp)

    # remove people account with @
    temp = re.sub(r'@\S+', "", temp)
    
    return temp

In [7]:
df_train['text']=df_train.text.apply(lambda x:func_textcleaning(x))
df_test['text']=df_test.text.apply(lambda x:func_textcleaning(x))

# Preprocessing

In [8]:
# Create tokenizer function for preprocessing
def spacy_tokenizer(text):

    # Define stopwords, punctuation, and numbers
    stop_words = spacy.lang.en.stop_words.STOP_WORDS
    punctuations = string.punctuation
    # numbers = "0123456789"

    # Create spacy object
    mytokens = sp(text)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    
    # Remove all word with less that 3 letters (remove noise)
    mytokens = [ word for word in mytokens if len(word)>2 ]

    # Return preprocessed list of tokens
    return mytokens

In [10]:
# Tokenize texts
%%time
processed_texts = []
for text in df_train.text:
    processed_text = spacy_tokenizer(text)
    processed_texts.append(processed_text)

# MODELS
# Initial Classification Model: 

Our initial assumption is that the bydault predictors (keyword, location and especially text) have a correlation or perhaps a causality with the fact that a tweet is either real or not

# TF-IDF with Logistic Regression

In [11]:
# Select features
X = df_train['text'] # the features we want to analyze
y = df_train['target'] # the labels, or answers, we want to test against

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [12]:
%%time
# Define vectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 1), tokenizer=spacy_tokenizer)

# Define classifier
classifier = LogisticRegressionCV(solver='lbfgs', max_iter=1000, cv=5)

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)

CPU times: user 1min 2s, sys: 112 ms, total: 1min 2s
Wall time: 1min 2s


In [13]:
#Train Accuracy  
%%time
pipe.score(X_train, y_train)

0.9289026275115919

In [14]:
# Predictions and evaluation for the model
%%time
y_pred = pipe.predict(X_test)

# Evaluate model
print(round(accuracy_score(y_test, y_pred), 4))

0.7876
CPU times: user 12.6 s, sys: 3.01 ms, total: 12.6 s
Wall time: 12.6 s


Decision Tree Model 

# SUBMISSION

In [15]:
# Create pipeline
pipe = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier)])

In [16]:
#fit with the train data: features and labels.
pipe.fit(df_train.text, df_train.target)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                                 tokenizer=<function spacy_tokenizer at 0x7f6e8fbe8bf8>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegressionCV(Cs=10, class_weight=None, cv=5,
                                      dual=Fal

In [17]:
#Train Accuracy  
pipe.score(df_train.text, df_train.target)

0.923659403492505

In [18]:
#text label predictions:
preds=pipe.predict(df_test.text)

In [19]:
#Data frame and submission format
y_pred=pd.DataFrame(preds)
y_pred.columns=['target']

In [20]:
from google.colab import files
y_pred.to_csv('coop22Nov.csv', index=False) 
files.download('coop22Nov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_train_copy.tail(5)

In [ ]:
df_train.text.nunique()

In [ ]:
df_train.keyword.value_counts()

In [ ]:
df_train.keyword.unique()